In [ ]:
import os
import numpy as np
from tqdm import tqdm
import hyperspy.api as hs

# Define paths
path = 'E:/'
Pt_dir = f'{path}/BOL-HAADF_result-model_haadf'
result_dir = f'{path}/BOL-Pt-mapping'

ionomer_dir = f'{path}/BOL-seg/ionomer-median'
support_dir = f'{path}/BOL-seg/support-median'

# Make output folders
os.makedirs(f"{result_dir}/Pt-ionomer", exist_ok=True)
os.makedirs(f"{result_dir}/Pt-carbon", exist_ok=True)
os.makedirs(f"{result_dir}/Pt-contact", exist_ok=True)

# File list
ionomer_lst = sorted(os.listdir(ionomer_dir))
support_lst = sorted(os.listdir(support_dir))
Pt_lst = sorted([file for file in os.listdir(Pt_dir) if file.endswith(".tif")])

# Open output text file
txt_path = f"{result_dir}/BOL-statistical_analysis.txt"
txt = open(txt_path, 'w')
txt.writelines('ionomer, support, ionomer+support, total area, only ionomer, only support, Pt, Pt-ionomer, Pt-support, Pt contact\n')

# Main loop
for i in tqdm(range(len(Pt_lst)), desc="Processing images"):
    ionomer = hs.load(os.path.join(ionomer_dir, ionomer_lst[i])).data
    support = hs.load(os.path.join(support_dir, support_lst[i])).data
    Pt = hs.load(os.path.join(Pt_dir, Pt_lst[i])).data

    # Upscale to match HAADF resolution
    ionomer = np.kron(ionomer, np.ones((8, 8)))
    support = np.kron(support, np.ones((8, 8)))
    ionomer = (ionomer >= 1).astype(np.uint8)
    support = (support >= 1).astype(np.uint8)
    Pt = (Pt >= 1).astype(np.uint8)

    area_seg = (ionomer * 3 + support * 2)

    # Overlap computations
    i_p = np.where(Pt * ionomer == 1, 255, Pt * 128)
    c_p = np.where(Pt * support == 1, 255, Pt * 128)
    a_p = np.where(Pt * ionomer * support == 1, 255, Pt * 128)

    # Save images
    hs.signals.Signal2D(i_p.astype(np.uint8)).save(f"{result_dir}/Pt-ionomer/{Pt_lst[i][:-4]}_Pt-ionomer.tif")
    hs.signals.Signal2D(c_p.astype(np.uint8)).save(f"{result_dir}/Pt-carbon/{Pt_lst[i][:-4]}_Pt-carbon.tif")
    hs.signals.Signal2D(a_p.astype(np.uint8)).save(f"{result_dir}/Pt-contact/{Pt_lst[i][:-4]}_Pt-contact.tif")

    # Write statistics
    txt.writelines(f'{ionomer.sum()}, {support.sum()}, {(area_seg == 5).sum()}, {(area_seg > 0).sum()}, '
                   f'{(area_seg == 3).sum()}, {(area_seg == 2).sum()}, {Pt.sum()}, '
                   f'{(i_p == 255).sum()}, {(c_p == 255).sum()}, {(a_p == 255).sum()}\n')

txt.close()
print(f"✅ Finished. Results saved to: {result_dir}")
